# 作業工程計画
2022.12.27作成<br>
1. 採点者品質予測_22冬納品_整形済み学習データを使用。21年と22夏データ。に22冬データをくっつけた
2. 設問情報を含む科目別データフレームと、設問情報を含まない科目混合データフレームの4種類を作成

reference:https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

# ライブラリ

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import japanize_matplotlib
import seaborn as sns
import re 
import scipy as sp
import time

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report #2値分類評価指標を出力

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

## 村上さん pandas_tool

In [1]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

# エクセルデータの読み込み

In [4]:
#整形済みデータの読み込み

"""
1. 秋22受領データを読み込む
2.　必要な列名を統一する
前処理データと受領データをくっつけるか、くっつけてからまとめて前処理するかはデータ受領後に考える。
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/intermediatedata/採点者品質予測_22冬納品_整形済み学習データ_参加回数修正済み_v1.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

df = []
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,
                   usecols=['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '身分', '年齢',
                            '偏差値', '登録試験点数', '参加回数', '完全一致率', 'Error', 'data'])#
df_raw = df.copy()

In [5]:
df_raw.columns.values

array(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '身分', '年齢', '偏差値',
       '登録試験点数', '参加回数', '完全一致率', 'Error', 'data'], dtype=object)

In [6]:
df_raw.dtypes

スタッフコード      int64
年度           int64
採点回         object
学年         float64
科目          object
分野          object
身分          object
年齢         float64
偏差値        float64
登録試験点数       int64
参加回数         int64
完全一致率      float64
Error        int64
data         int64
dtype: object

In [7]:
len(df_raw)

24850

# データ前処理

In [8]:
def rem_error(df_input):
    df = df_input.copy()
    df = df[df['Error']==0]
    return df

In [9]:
def pret_data_keiken(df_input):
    """
    同一採点者を取得
    完全一致率の欠けているレコードを削除
    一つ前の参加回数に、今回の参加回数を結合
    """
    df = df_input.copy()
    temp=[]
    s = list(sorted(df['スタッフコード'].unique().astype(int)))#スタッフコード単位で繰り返す
    for h in s:
        df1 = df[df['スタッフコード']==h].loc[:,['スタッフコード','科目','参加回数','分野','年度','採点回','学年',
                                          '登録試験点数','身分', '年齢', '偏差値','完全一致率','data']]
        df1 = df1[df1['完全一致率']!= 0]#完全一致率が0でないデータ
        df1 = df1.sort_values('参加回数', ascending=True)#参加回数で並びかえる
        df1['No']= range(1, len(df1) + 1)#同一スタッフコードの人の、完全一致率をもつレコードに連番をふる
    
        n = list(sorted(df1['No'].unique().astype(int)))
    
        for i in n:
            df2 = df1.loc[:,['スタッフコード','参加回数','分野','完全一致率','No']]
            df2 = df2[df2['No']== i]
            df2 = df2.rename(columns = {'参加回数':'前回の参加回数',
                                    '分野':'前回の分野',
                                    '完全一致率':'前回の完全一致率'})
            df2['前回の参加回数'] = df2['前回の参加回数'].astype(int)
            df2 = df2.drop(columns = 'No')
        
            df3 = df1.loc[:,['スタッフコード','科目','参加回数','分野','年度','採点回','学年','完全一致率',
                        '登録試験点数','身分', '年齢', '偏差値','data','No']]
            df3 = df3[df3['No']== (i+1)]
            df3 = df3.rename(columns = {'参加回数':'今回の参加回数',
                                    '分野':'今回の分野',                                    
                                    '採点回':'今回の採点回',
                                    '年度':'今回の年度',
                                    '完全一致率':'今回の完全一致率'})
            df3['今回の参加回数'] = df3['今回の参加回数'].astype(int)
            df3 = df3.drop(columns = 'No')
        
            ret = pd.merge(df2,df3,on = 'スタッフコード',how = 'inner')
            temp.append(ret)
    dfs = temp[0] #集計結果を一つのDFに結合
    for i in temp[1:]:
        dfs = dfs.append(i)
    dfs = dfs.dropna(subset=['前回の完全一致率'])
    #df = []
    #df = dfs.drop_duplicates()
    
    return dfs

In [10]:
def pret_data_shinjin(df_input):#修正
    """
    新人のデータだけ抽出
    """
    df = df_input.copy()
    df = df.loc[:,['スタッフコード','参加回数','科目','分野','年度','採点回','学年','完全一致率','登録試験点数','身分', '年齢', '偏差値','data']]
    df = df[df['参加回数']==0] 
    return df

In [48]:
def pret_data_etc(df_input):#経験者だが、過去データに前回のデータがない採点者
    """
    経験者だが、過去データに前回のデータがない採点者を抽出
    """
    df = df_input.copy()
    df = df.loc[:,['スタッフコード','参加回数','科目','分野','年度','採点回','学年','完全一致率','登録試験点数','身分', '年齢', '偏差値','data']]
    #スタッフコードの重複があるデータを全て削除
    df = df.drop_duplicates(subset='スタッフコード',keep=False)
    #新人データを削除
    df = df[~(df['参加回数']==0)] 
    return df

# 前処理済みデータの出力

In [11]:
# 時間計測開始
time_sta = time.time()

# 処理開始

#前処理したデータ
df_pret = rem_error(df_raw)

#機械学習に使うデータ_新人用
df_pret_shinjin = pret_data_shinjin(df_pret)

#機械学習に使うデータ_経験者用
df_pret_keiken = pret_data_keiken(df_pret)

#機械学習に使うデータ_経験者だが過去データがない
df_pret_etc = pret_data_etc(df_pret)

# 時間計測終了
time_end = time.time()
# 経過時間（秒）
tim = time_end- time_sta

print(tim)

125.88842797279358


### データ数チェック

In [53]:
#新人
df = df_pret_shinjin
print(len(df),len(df[df['data']==0]),len(df[df['data']==1]),len(df[df['data']==2]))

4259 2363 748 1148


In [54]:
#経験
df = df_pret_keiken
print(len(df),len(df[df['data']==0]),len(df[df['data']==1]),len(df[df['data']==2]))

7622 4186 1679 1757


In [52]:
#経験　過去データなし
print(len(df),len(df[df['data']==0]),len(df[df['data']==1]),len(df[df['data']==2]))

1969 929 152 888


In [12]:
"""
Excelの書き出し 
"""
name = "採点者品質予測_22冬納品_前処理済み学習データ_v1"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df_raw.to_excel(writer, sheet_name='整形済みデータ',encoding='utf-8-sig', index = False)
    df_pret.to_excel(writer, sheet_name='前処理済みデータ',encoding='utf-8-sig', index = False)
    df_pret_shinjin.to_excel(writer, sheet_name='機械学習用データ_新人',encoding='utf-8-sig', index = False)
    df_pret_keiken.to_excel(writer, sheet_name='機械学習用データ_経験者',encoding='utf-8-sig', index = False)

In [14]:
#N数の確認
df = df_pret_shinjin
df.dropna(subset=['偏差値'], inplace=True)
print('新人のサンプル数{}'.format(str(len(df))))

新人のサンプル数4259


In [13]:
#N数の確認
df = df_pret_keiken
df.dropna(subset=['偏差値'], inplace=True)
print('経験者の参加回数別のサンプル数{}'.format(str(len(df))))
for i in range(1,6):
    print('参加経験{}回のサンプル数{}'.format(str(i),str(len(df[df['今回の参加回数']==i]))))
print('参加経験6回以上のサンプル数{}'.format(str(len(df[df['今回の参加回数']>=6]))))

経験者の参加回数別のサンプル数7622
参加経験1回のサンプル数1416
参加経験2回のサンプル数1133
参加経験3回のサンプル数1057
参加経験4回のサンプル数1040
参加経験5回のサンプル数813
参加経験6回以上のサンプル数2163


## モデル作成

### DecisionTreeRegressorメソッドの概要

|引数名|概要|デフォルト|
| :---- | :---- | :---- |
|criterion|不純度を測定する基準（平均二乗誤差、平均絶対誤差など）|‘mse’|
|splitter|条件探索アルゴリズムを選択するオプション（’best’と’rondom’が指定可能）|‘best’|
|max_depth|決定木のノード深さの制限値。ツリーが深くなりすぎて過学習の状態に陥った際は、このパラメータが正則化の役割を果たす。|None|
|min_samples_split|ノードを分割するために必要なサンプルの最小値|2|
|min_samples_leaf|1ノードの深さを作成するために必要となるデータ数の最小値。指定した値以上のデータ数を持たないノードは作られない。|1|
|min_weight_fraction_leaf|サンプルの重みを考慮した上でのmin_samples_leafに該当|0.0|
|max_features|ランダムに指定する説明変数の数(全ての説明変数がモデル学習に活用されるわけではなく、ランダムに割り振られる）|None|
|random_state|乱数シード|None|
|max_leaf_nodes|作成される決定木の葉の数を、指定した値以下に制御する|None|
|min_impurity_decrease|決定木の成長の早期停止するための閾値。不純度が指定の値より減少した場合、ノードを分岐し、不純度が指定の値より減少しなければ分岐を抑制。|0.0|
|ccp_alpha|ccp_alphaが大きいほどプルーニングされるノードの数が増加。プルーニングとは、精度低下をできるだけ抑えながら過剰な重みを排除するプロセスを指す。|0.0|


In [15]:
"""
https://stackoverflow.com/questions/50607740/reverse-a-get-dummies-encoding-in-pandas
ダミー変数を元に戻す。エクセル出力用
"""

def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [16]:
#機械学習に使うデータの前処理_新人用
df_pret_shinjin.columns.values

array(['スタッフコード', '参加回数', '科目', '分野', '年度', '採点回', '学年', '完全一致率',
       '登録試験点数', '身分', '年齢', '偏差値', 'data'], dtype=object)

In [17]:
def make_data_shinjin(df_input):
    
    """
    学習,検証,予測データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    df_pred = df[df['data']==2].copy()
    
    #rem_y = '参加回数'
    use_y = '完全一致率'

    #1.不要な変数を削除
    #df = df.drop(columns = rem_y)

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['分野','身分'])
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = use_y)#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,[use_y,'data']]#目的変数と選択用変数だけ含むデータフレーム　
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[df_X['data']==0]
    #訓練用目的変数
    y_train = df_y[df_y['data']==0]

    """22夏が検証用"""
    #説明変数
    X_test = df_X[df_X['data']==1]
    #目的変数
    y_test = df_y[df_y['data']==1]

    """予測用"""
    #説明変数
    X_pred = df_X[df_X['data']==2]#秋に変更するところ
    
    #5.不要な変数を削除
    rem_cols_x = ["スタッフコード","採点回","年度","科目","参加回数","data"]
    rem_cols_y = ["data"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_id =X_test.loc[:,['スタッフコード']]
    X_test_kamoku =X_test.loc[:,['科目']]
    X_test = X_test.drop(columns = rem_cols_x)
    X_pred = X_pred.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku, X_pred, df_pred]

    return df_res

In [11]:
def make_data_shinjin(df_input):#オリジナル
    
    """
    学習,検証,予測データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    #0の場合、完全一致率を予測。　1の場合、最終ペース相対値を予測
    rem_y = '参加回数'
    use_y = '完全一致率'

    #1.不要な変数を削除
    df = df.drop(columns = rem_y)

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = use_y)#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,[use_y,'data']]#目的変数と選択用変数だけ含むデータフレーム　
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==2022))]#秋に変更するところ
    #訓練用目的変数
    y_train = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==2022))]#秋に変更するところ

    """22夏が検証用"""
    #訓練用説明変数
    X_test = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==2022)]#秋に変更するところ
    #訓練用目的変数
    y_test = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==2022)]#秋に変更するところ
    
    #5.不要な変数を削除
    rem_cols_x = ["スタッフコード","採点回","年度","科目","参加回数","data"]
    rem_cols_y = ["data"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_id =X_test.loc[:,['スタッフコード']]
    X_test_kamoku =X_test.loc[:,['科目']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku]

    return df_res

In [18]:
df_pret_keiken.columns.values

array(['スタッフコード', '前回の参加回数', '前回の分野', '前回の完全一致率', '科目', '今回の参加回数',
       '今回の分野', '今回の年度', '今回の採点回', '学年', '今回の完全一致率', '登録試験点数', '身分', '年齢',
       '偏差値', 'data'], dtype=object)

In [20]:
def make_data_keiken(df_input):
    
    """
    学習,検証,予測データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    df_pred = df[df['data']==2].copy()
    df_pred = df_pred.rename(columns = {'今回の採点回':'採点回',#0始まりで数えているため、
                             '今回の年度':'年度',
                             '今回の参加回数':'参加回数',
                             '今回の分野':'分野'})
    rem_y = ['前回の分野','前回の参加回数']
    use_y = '今回の完全一致率'

    
    #1.不要な変数を削除
    df = df.drop(columns = rem_y)
    df = df.rename(columns = {'今回の採点回':'採点回',#0始まりで数えているため、
                             '今回の年度':'年度',
                             '今回の参加回数':'参加回数',
                             '今回の分野':'分野'})

    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['分野','身分'])
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = use_y)#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['data',use_y]]#目的変数と選択用変数だけ含むデータフレーム　#秋に変更するところ
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[df_X['data']==0]#秋に変更するところ
    #訓練用目的変数
    y_train = df_y[df_y['data']==0]#秋に変更するところ

    """22夏が検証用"""
    #説明変数
    X_test = df_X[df_X['data']==1]#秋に変更するところ
    #目的変数
    y_test = df_y[df_y['data']==1]#秋に変更するところ

    """予測用"""
    #説明変数
    X_pred = df_X[df_X['data']==2]#秋に変更するところ
    
    #5.不要な変数を削除
    rem_cols_x = ["スタッフコード","採点回","年度","科目","data"]
    rem_cols_y = ["data"]#秋に変更するところ

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_id =X_test.loc[:,['スタッフコード']]
    X_test_kamoku =X_test.loc[:,['科目']]
    X_test = X_test.drop(columns = rem_cols_x)
    X_pred = X_pred.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku, X_pred, df_pred]

    return df_res

In [21]:
"""
モデル作成
"""
def train_model(df,a=2,b=1,c=None):
    X_train= df[0]
    y_train= df[1]
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=0,#出力結果の固定のため 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    return model

In [22]:
def get_test(df,model):
    """
    モデルを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
    """
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    X_test_id = df[4]
    X_test_kamoku = df[5]
    y_pred  = model.predict(X_test)
    
    #得た結果を学習データとマージしてデータフレームで返す
    
    df_test=[]
    df_test = undummify(X_test)  #企画ペースを入れるためにここをいじった。元はX_test
    df_test.loc[:,'スタッフコード']= X_test_id
    df_test.loc[:,'科目']= X_test_kamoku
    df_test.loc[:,'完全一致率']= y_test
    df_test.loc[:,'AI想定完全一致率']= y_pred #上のデータに予測値をマージ
    #学習データの同じ学年分野カテゴリの中の基礎統計量
    df_train = undummify(X_train)
    df_train.loc[:,'完全一致率'] = y_train
    df_temp = df_train.groupby(['分野','学年'],as_index=False).agg(
        学習データ_N数=pd.NamedAgg(column="完全一致率", aggfunc="count"),
        学習データ_平均値=pd.NamedAgg(column="完全一致率", aggfunc="mean"),
        学習データ_中央値=pd.NamedAgg(column="完全一致率", aggfunc="median"),
        学習データ_最大値=pd.NamedAgg(column="完全一致率", aggfunc="max"),
        学習データ_最小値=pd.NamedAgg(column="完全一致率", aggfunc="min"))
    df_res = pd.merge(df_test, df_temp, on=['分野', '学年'], how='left')
    # 実測値_完全一致率をランク分け
    x = "完全一致率"
    conditions = [
            (df_res[x] >= 0.95),
            (df_res[x] >= 0.7)
             ]
    choices = ["0.95~","0.70~0.95"]
    df_res.loc[:,'実測一致率ランク'] = np.select(conditions, choices, default = "~0.70")

        # AI想定完全一致率をランク分け 評価用
    x = "AI想定完全一致率"
    conditions = [
            (df_res[x] >= 0.95),
            (df_res[x] >= 0.7)
             ]
    choices = ["0.95~","0.70~0.95"]
    df_res.loc[:,'想定一致率ランク'] = np.select(conditions, choices, default = "~0.70")
    
    return df_res

In [23]:
def get_pred(df,model):
    """
    モデルを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
    """
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    X_test_id = df[4]
    X_test_kamoku = df[5]
    X_pred = df[6]
    df_pred = df[7]
    y_pred  = model.predict(X_pred)
    
    #得た結果を学習データとマージしてデータフレームで返す
    
    df_test=[]
    df_test = undummify(df_pred)  #企画ペースを入れるためにここをいじった。元はX_test
    df_test.loc[:,'AI想定完全一致率']= y_pred #上のデータに予測値をマージ
    #学習データの同じ学年分野カテゴリの中の基礎統計量
    df_train = undummify(X_train)
    df_train.loc[:,'完全一致率'] = y_train
    df_temp = df_train.groupby(['分野','学年'],as_index=False).agg(
        学習データ_N数=pd.NamedAgg(column="完全一致率", aggfunc="count"),
        学習データ_平均値=pd.NamedAgg(column="完全一致率", aggfunc="mean"),
        学習データ_中央値=pd.NamedAgg(column="完全一致率", aggfunc="median"),
        学習データ_最大値=pd.NamedAgg(column="完全一致率", aggfunc="max"),
        学習データ_最小値=pd.NamedAgg(column="完全一致率", aggfunc="min"))
    df_res = pd.merge(df_test, df_temp, on=['分野', '学年'], how='left')
    
    return df_res

In [22]:
def get_importance(df,model):
    """
    変数の重要度を返す
    """
    X_test = df[2]
    feature = model.feature_importances_
    label = X_test.columns
    indices = np.argsort(feature)

    # 特徴量の重要度の棒グラフ
    fig =plt.figure (figsize = (10,10))
    plt.ion()
    plt.barh(range(len(feature)), feature[indices])
    plt.yticks(range(len(feature)), label[indices], fontsize=14)
    plt.xticks(fontsize=14)
    plt.ylabel("Feature", fontsize=18)
    plt.xlabel("Feature Importance", fontsize=18)
    plt.tight_layout()
    plt.close(fig)
    return fig

In [23]:
#使用ライブラリ　樹形図作成
# from sklearn import tree
# import graphviz
def make_treegraph(df,model):
    """
    樹形図を作成し、返す
    """
    X_train= df[0]
    X_train_feature_names = X_train.columns.values.tolist()
    dot_data = tree.export_graphviz(model, out_file=None,
                                    feature_names= X_train_feature_names,
                                    # class_names=iris.target_names,
                                    filled=True, rounded=True, special_characters=True
                                    )
    treegraph = graphviz.Source(dot_data) 
    return treegraph

## 採点者別品質予測

In [24]:
#検証用
def get_result_test(df_input,case=int):
    df = df_input.copy()
    df_temp=[]#[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku]
    model = []
    pred = []
    #新人モデルの結果
    if case == 0:
        df_temp = make_data_shinjin(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_test(df_temp, model)
    #経験者モデルの結果
    elif case == 1:
        df_temp = make_data_keiken(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_test(df_temp, model)
            
    return pred

In [25]:
#予測用
def get_result_pred(df_input,case=int):
    df = df_input.copy()
    df_temp=[]#[X_train,y_train,X_test,y_test,X_test_id,X_test_kamoku]
    model = []
    pred = []
    #新人モデルの結果
    if case == 0:
        df_temp = make_data_shinjin(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_pred(df_temp, model)
    #経験者モデルの結果
    elif case == 1:
        df_temp = make_data_keiken(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        pred = get_pred(df_temp, model)
            
    return pred

In [55]:
#結果出力
target_names = ['0.70~0.95', '~0.70', '0.95~']
a=['実測一致率ランク','想定一致率ランク']#y_true, y_pred
df1 = get_result_test(df_pret_shinjin, case = 0)#新人モデルの検証結果
df2 = get_result_test(df_pret_keiken, case = 1)#経験モデルの検証結果
df3 = get_result_test(df_pret_etc, case = 0)#経験（過去データなし）モデルの検証結果 新人モデルと同じ学習法で、データセットだけ変えてある

df1_pred = get_result_pred(df_pret_shinjin, case = 0)#新人モデルの予測結果
df2_pred = get_result_pred(df_pret_keiken, case = 1)#経験モデルの予測結果
df3_pred = get_result_pred(df_pret_etc, case = 0)#経験（過去データなし）モデルの予測結果


name_file = "採点者品質予測_22冬_予測値_1227_v2" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df1_pred.to_excel(writer, sheet_name='予測_新人_分野',encoding='utf-8-sig', index = False)
    df2_pred.to_excel(writer, sheet_name='予測_経験_分野',encoding='utf-8-sig', index = False)
    df3_pred.to_excel(writer, sheet_name='予測_経験過去無し_分野',encoding='utf-8-sig', index = False)
    df1.to_excel(writer, sheet_name='22夏検証_新人_分野',encoding='utf-8-sig', index = False)
    df2.to_excel(writer, sheet_name='22夏検証_経験_分野',encoding='utf-8-sig', index = False)
    df3.to_excel(writer, sheet_name='22夏検証_経験過去無し_分野',encoding='utf-8-sig', index = False)
    pd.DataFrame(classification_report(y_true = list(df1[a[0]]),
                                       y_pred = list(df1[a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='22夏検証_新人_分野_score',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df2[a[0]]),
                                       y_pred = list(df2[a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='22夏検証_経験_分野_score',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df3[a[0]]),
                                       y_pred = list(df3[a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='22夏検証_経験過去無し_分野_score',encoding='utf-8-sig', index = True)

##　予測値を納品用に整形

#### 受領したデータを型にして、新人モデルと予測モデルの結果をスタッフコードで結合する。

In [56]:
#整形済みデータの読み込み

"""
納品用に、受領データ（分野情報を補足したもの）を読み込む
"""
# 
path_name = "/Users/s.ogura/Documents/CRLEA/data/intermediatedata/採点者品質予測_22冬納品_整形済み学習データ_参加回数修正済み_v1.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names


df = []
df = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)
#usecols=['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '身分', '年齢','偏差値', '登録試験点数', '参加回数', '完全一致率', 'Error', 'data']
df = df[df['data']==2]
df_orijin = df.copy()

In [47]:
df_orijin.columns.values

array(['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学',
       '学部', '偏差値', '登録試験点数', '参加回数_修正前', '参加回数', '完全一致率', 'Error',
       'data'], dtype=object)

In [32]:
df1_pred.columns.values

array(['スタッフコード', '参加回数', '科目', '分野', '年度', '採点回', '学年', '完全一致率',
       '登録試験点数', '身分', '年齢', '偏差値', 'data', 'AI想定完全一致率', '学習データ_N数',
       '学習データ_平均値', '学習データ_中央値', '学習データ_最大値', '学習データ_最小値'], dtype=object)

In [33]:
df2_pred.columns.values

array(['スタッフコード', '前回の参加回数', '前回の分野', '前回の完全一致率', '科目', '参加回数', '分野',
       '年度', '採点回', '学年', '今回の完全一致率', '登録試験点数', '身分', '年齢', '偏差値', 'data',
       'AI想定完全一致率', '学習データ_N数', '学習データ_平均値', '学習データ_中央値', '学習データ_最大値',
       '学習データ_最小値'], dtype=object)

In [57]:
df3_pred.columns.values

array(['スタッフコード', '参加回数', '科目', '分野', '年度', '採点回', '学年', '完全一致率',
       '登録試験点数', '身分', '年齢', '偏差値', 'data', 'AI想定完全一致率', '学習データ_N数',
       '学習データ_平均値', '学習データ_中央値', '学習データ_最大値', '学習データ_最小値'], dtype=object)

In [58]:
"""
df1_pred　新人モデルの予測結果
df2_pred　経験モデルの予測結果
df3_pred　経験モデルの予測結果
各モデルの結合
"""
def select_res(df_input1, df_input2, df_input3):
    # colname= ['スタッフコード', '参加回数', 'AI想定完全一致率', '学習データ_N数','学習データ_平均値', '学習データ_中央値', '学習データ_最大値', '学習データ_最小値']
    colname= ['スタッフコード','AI想定完全一致率', '学習データ_N数','学習データ_平均値', '学習データ_中央値', '学習データ_最大値', '学習データ_最小値']
    df1 = df_input1.copy()
    df1 = df1[colname]
    
    df2 = df_input2.copy()
    df2 = df2[colname]
    
    df3 = df_input3.copy()
    df3 = df3[colname]
    
    df = df1.append(df2)
    df = df.append(df3)
    #df[colname] = df[colname].fillna('na')
    
    return df

df_pred_comb = select_res(df1_pred,df2_pred,df3_pred)

In [59]:
num_check ={"受領件数":len(df_orijin),
 "予測件数":len(df_pred_comb),
 "欠損件数":len(df_orijin)-len(df_pred_comb)}
print(num_check)

{'受領件数': 3887, '予測件数': 3793, '欠損件数': 94}


In [61]:
#受領した元データに予測値を結合
df1 = df_orijin
df2 = df_pred_comb
df = pandas_tool.merge(df1,df2, key = ["スタッフコード"],how = 'left')
colname= [ '参加回数','AI想定完全一致率', '学習データ_N数','学習データ_平均値', '学習データ_中央値', '学習データ_最大値', '学習データ_最小値']
df[colname] = df[colname].fillna('na')
collist= ['スタッフコード', '年度', '採点回', '学年', '科目', '分野', '割当', '身分', '年齢', '大学',
       '学部', '偏差値', '登録試験点数', '参加回数_修正前', '参加回数', '完全一致率',
         'AI想定完全一致率', '学習データ_N数', '学習データ_平均値',
       '学習データ_中央値', '学習データ_最大値', '学習データ_最小値', 
        'Error','data', ]
df=df[collist]
df_output = df.copy()

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:3887
df2のオブザベーション:3793
mergeのオブザベーション:3887


In [62]:
"""
Excelの書き出し 
"""
df = df_output

name = "採点者品質予測_22冬_納品_1227_v3"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df.to_excel(writer, sheet_name='予測値_採点者',encoding='utf-8-sig', index = False)

# ここまで

In [69]:
def get_graph(df_input,case=int):
    df = df_input.copy()
    path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
    #新人モデルの結果
    if case == 0:
        df_temp= make_data_shinjin(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        res = get_importance(df_temp, model)
        name = "採点者品質予測_22秋採点_重要度_新人_0906_v1"#ファイル名
        path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
        res.savefig('{p}/{n}.png'.format(p = path_folder, n = name), format="png", dpi=300)
        treegraph = make_treegraph(df_temp, model)
        name = "採点者品質予測_22秋採点_樹形図_新人_0906_v1".format(i)#ファイル名
        path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
        treegraph.render(path_file)

    #経験者モデルの結果
    elif case == 1:
        df_temp= make_data_keiken(df)
        model = train_model(df_temp,a=2,b=1,c=None)
        res = get_importance(df_temp, model)
        name = "採点者品質予測_22秋採点_重要度_新人_0906_v1"#ファイル名
        path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
        res.savefig('{p}/{n}.png'.format(p = path_folder, n = name), format="png", dpi=300)
        treegraph = make_treegraph(df_temp, model)
        name = "採点者品質予測_22秋採点_樹形図_新人_0906_v1".format(i)#ファイル名
        path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
        treegraph.render(path_file)

In [109]:
#重要度、樹形図出力
get_graph(df_pret_shinjin,case=0)
get_graph(df_pret_keiken,case=1)

# メモ

In [74]:
#クラス分類評価指標　作成メモ
from sklearn.metrics import classification_report
df=df1.copy()

#y_true = list(df[0]['実測一致率ランク'])#それぞれのランク
#y_pred = list(df[0]['想定一致率ランク2'])#それぞれのランク

target_names = ['0.70~0.95', '~0.70', '0.95~']
d = pd.DataFrame(classification_report(y_true = list(df[0]['実測一致率ランク']), y_pred = list(df[0]['想定一致率ランク2']),
                                       target_names=target_names,output_dict=True))

#print(classification_report(y_true, y_pred,target_names=target_names))

In [79]:
#結果出力
select_cols = ['AI想定完全一致率','完全一致率','実予誤差']
target_names = ['0.70~0.95', '~0.70', '0.95~']
a=['実測一致率ランク','想定一致率ランク2']#y_true, y_pred
df1 = get_result(df_pret_shinjin, case = 0)#新人モデルの予測結果
df2 = get_result(df_pret_keiken, case = 1)#経験モデルの予測結果



name_file = "採点者品質予測_科目混合_完全一致率_0902_v2" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df1[0].to_excel(writer, sheet_name='value_新人_分野',encoding='utf-8-sig', index = False)
    df1[1].to_excel(writer, sheet_name='value_新人_科目',encoding='utf-8-sig', index = False)
    df1[2].to_excel(writer, sheet_name='value_新人',encoding='utf-8-sig', index = False)
    df2[0].to_excel(writer, sheet_name='value_経験_分野',encoding='utf-8-sig', index = False)
    df2[1].to_excel(writer, sheet_name='value_経験_科目',encoding='utf-8-sig', index = False)
    df2[2].to_excel(writer, sheet_name='value_経験',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df1[0].loc[:,select_cols])[0].to_excel(writer, sheet_name='summary_新人_分野',encoding='utf-8-sig', index = True)
    pandas_tool.summary(df1[1].loc[:,select_cols])[0].to_excel(writer, sheet_name='summary_新人_科目',encoding='utf-8-sig', index = True)
    pandas_tool.summary(df1[2].loc[:,select_cols])[0].to_excel(writer, sheet_name='summary_新人',encoding='utf-8-sig', index = True)
    pandas_tool.summary(df2[0].loc[:,select_cols])[0].to_excel(writer, sheet_name='summary_経験_分野',encoding='utf-8-sig', index = True)
    pandas_tool.summary(df2[1].loc[:,select_cols])[0].to_excel(writer, sheet_name='summary_経験_科目',encoding='utf-8-sig', index = True)
    pandas_tool.summary(df2[2].loc[:,select_cols])[0].to_excel(writer, sheet_name='summary_経験',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df1[0][a[0]]),
                                       y_pred = list(df1[0][a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='score_新人_分野',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df1[1][a[0]]),
                                       y_pred = list(df1[1][a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='score_新人_科目',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df1[2][a[0]]),
                                       y_pred = list(df1[2][a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='score_新人',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df2[0][a[0]]),
                                       y_pred = list(df2[0][a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='score_経験_分野',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df2[1][a[0]]),
                                       y_pred = list(df2[1][a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='score_経験_科目',encoding='utf-8-sig', index = True)
    pd.DataFrame(classification_report(y_true = list(df2[2][a[0]]),
                                       y_pred = list(df2[2][a[1]]),
                                       target_names=target_names,output_dict=True)).to_excel(writer, sheet_name='score_経験',encoding='utf-8-sig', index = True)

In [28]:
def get_graph(df_input,case=int):
    df = df_input.copy()
    path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
    #新人モデルの結果
    if case == 0:
        dfs=[[],[],[]]
        dfs[0] = make_data_shinjin(df,val=0)
        dfs[1] = make_data_shinjin2(df,val=0)
        dfs[2] = make_data_shinjin3(df,val=0)
        models = [[],[],[]]
        for i in range(0,len(dfs)):
            model = train_model(dfs[i],a=2,b=1,c=None)
            res = get_importance(dfs[i], model)
            name = "採点者品質予測_科目混合_完全一致率_重要度_新人{}_0902_v1".format(i)#ファイル名
            path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
            res.savefig('{p}/{n}.png'.format(p = path_folder, n = name), format="png", dpi=300)
            treegraph = make_treegraph(dfs[i], model)
            name = "採点者品質予測_科目混合_完全一致率_樹形図_新人{}_0902_v1".format(i)#ファイル名
            path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
            treegraph.render(path_file)

    #経験者モデルの結果
    elif case == 1:
        dfs=[[],[],[]]
        dfs[0] = make_data_keiken(df,val=0)
        dfs[1] = make_data_keiken2(df,val=0)
        dfs[2] = make_data_keiken3(df,val=0)
        models = [[],[],[]]
        for i in range(0,len(dfs)):
            model = train_model(dfs[i],a=2,b=1,c=None)
            res = get_importance(dfs[i], model)
            name = "採点者品質予測_科目混合_完全一致率_重要度_経験{}_0902_v1".format(i)#ファイル名
            path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
            res.savefig('{p}/{n}.png'.format(p = path_folder, n = name), format="png", dpi=300)
            treegraph = make_treegraph(dfs[i], model)
            name = "採点者品質予測_科目混合_完全一致率_樹形図_経験{}_0902_v1".format(i)#ファイル名
            path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
            treegraph.render(path_file)
            
    return